In [17]:
import numpy as np
import os
import spacy
import scispacy
nlp = spacy.load("en_core_sci_sm")
def word2vec(x):
    return nlp(x).vector

<h1>Importing Proteins and Common English Words</h1>

In [18]:


if os.path.isfile('dataset.npy'):
    dataset = np.load('dataset.npy', allow_pickle=True)
    proteinVectors = np.load('proteinVectors.npy', allow_pickle=True)
    commonVectors = np.load('commonVectors.npy', allow_pickle=True)
else:
    with open ("protein_list.txt", "r", encoding="utf8") as myfile:
        lines=myfile.readlines()
        myfile.close()

    proteins = [l.rstrip() for l in lines]    
    print(len(proteins),"proteins imported")
    
    import nltk
    nltk.download('words')
    from nltk.corpus import words
    word_list = words.words()
    # prints 236736


    words = [w for w in word_list]
    print(len(words),"words imported")
    
    
    for p in proteins:
        if p in words:
            words.remove(p)
            
    proteinVectors = [word2vec(p) for p in proteins]
    
    commonVectors = [word2vec(w) for w in words]

    dataset = [(p,1) for p in proteinVectors]    
    print("done with converting proteins to vectors")
    
    dataset += [(w,0) for w in commonVectors]
    print("done with converting words to vectors")

    np.save('dataset.npy', dataset)
    np.save('proteinVectors.npy', proteinVectors)
    np.save('commonVectors.npy', commonVectors)


In [19]:
import random

#print(dataset[0])

np.random.shuffle(dataset)

#print(dataset[0])

#dataset = random.sample(dataset,len(dataset))





vectors = [l[0] for l in dataset]
labels = [l[1] for l in dataset]

print(len(vectors))
print(len(labels))
print(round(len(labels)/5))
ratio = round(len(labels)/5)

print(vectors[0].shape)

for i,k in enumerate(labels):
    print(i,k)
    if i>10:
        break


443908
443908
88782
(96,)
0 0
1 0
2 0
3 0
4 0
5 0
6 1
7 0
8 0
9 0
10 0
11 1


In [20]:
testSet = np.array(vectors[:ratio])
testLabel = labels[:ratio]

trainSet = np.array(vectors[ratio:])
trainLabel = labels[ratio:]

print(len(testSet))
print(len(trainSet))
print(type(trainSet))

88782
355126
<class 'numpy.ndarray'>


In [21]:
for t in trainSet:
    print(t.shape)
    break

(96,)


In [22]:
vectorSize = trainSet[0].shape[0]
print(vectorSize)

96


In [23]:
import os
import re
import gc
import time
import numpy as np
import tensorflow as tf
import pandas as pd

from numpy import array
from numpy import asarray
from numpy import zeros

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend


from keras import regularizers

#from keras.models import Sequential, Model

from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import Conv2D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Reshape
from keras.layers import MaxPooling1D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling1D
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Lambda
from keras.layers.merge import concatenate


from keras.layers.advanced_activations import LeakyReLU

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from keras.optimizers import SGD, Adam, RMSprop






print("Imports work")

Imports work


<h1>Entity Recognition Training</h1>

In [24]:

Dense1Results = []
Dense2Results = []
Dense3Results = []


all_epochs_to_run  = 1
opt = 'adam'
learningRate = 0.1
epochs_to_run = all_epochs_to_run

lossType = 'binary_crossentropy'#"sparse_categorical_crossentropy" #"categorical_crossentropy" #'binary_crossentropy'

In [25]:
epochs_to_run = 100

In [26]:
input_layer = Input(shape=(vectorSize,))

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

#x = Dense(500, activation='sigmoid')(input_layer)

x = Dense(100, activation='sigmoid')(input_layer)

x = Dense(50, activation='sigmoid')(input_layer)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp1 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp1.summary()


#compile
mlp1.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp1.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp1.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp1.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp1.predict(np.array([word2vec(word)]))


print(prediction)


Dense1Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                4850      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 51        
Total params: 4,901
Trainable params: 4,901
Non-trainable params: 0
_________________________________________________________________

Starting training...

Epoch 1/100
355126/355126 [==============================] - 22s 61us/step - loss: 0.0661 - accuracy: 0.9790
Epoch 2/100
355126/355126 [==============================] - 22s 61us/step - loss: 0.0484 - accuracy: 0.9853
Epoch 3/100
355126/355126 [==============================] - 22s 63us/step - loss: 0.0452 - accuracy: 0.9863
Epoch 4/100
355126/355126 [==========

In [27]:
epochs_to_run = 200

In [28]:
input_layer = Input(shape=(vectorSize,))

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

#x = Dense(500, activation='sigmoid')(input_layer)

x = Dense(100, activation='sigmoid')(input_layer)

x = Dense(50, activation='sigmoid')(input_layer)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp2 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp2.summary()


#compile
mlp2.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp2.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp2.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp2.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp2.predict(np.array([word2vec(word)]))


print(prediction)


Dense2Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 50)                4850      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 51        
Total params: 4,901
Trainable params: 4,901
Non-trainable params: 0
_________________________________________________________________

Starting training...

Epoch 1/200
355126/355126 [==============================] - 22s 62us/step - loss: 0.0663 - accuracy: 0.9790
Epoch 2/200
355126/355126 [==============================] - 21s 60us/step - loss: 0.0487 - accuracy: 0.9853
Epoch 3/200
355126/355126 [==============================] - 21s 60us/step - loss: 0.0450 - accuracy: 0.9864
Epoch 4/200
355126/355126 [==========

355126/355126 [==============================] - 21s 59us/step - loss: 0.0247 - accuracy: 0.9925
Epoch 143/200
355126/355126 [==============================] - 21s 60us/step - loss: 0.0246 - accuracy: 0.9926
Epoch 144/200
355126/355126 [==============================] - 21s 59us/step - loss: 0.0245 - accuracy: 0.9925
Epoch 145/200
355126/355126 [==============================] - 21s 59us/step - loss: 0.0246 - accuracy: 0.9925
Epoch 146/200
355126/355126 [==============================] - 21s 59us/step - loss: 0.0246 - accuracy: 0.9925
Epoch 147/200
355126/355126 [==============================] - 21s 59us/step - loss: 0.0245 - accuracy: 0.9925
Epoch 148/200
355126/355126 [==============================] - 21s 60us/step - loss: 0.0244 - accuracy: 0.9924
Epoch 149/200
355126/355126 [==============================] - 21s 60us/step - loss: 0.0244 - accuracy: 0.9926
Epoch 150/200
355126/355126 [==============================] - 21s 60us/step - loss: 0.0244 - accuracy: 0.9926
Epoch 151/200
3

In [29]:
epochs_to_run = 300

In [30]:
input_layer = Input(shape=(vectorSize,))

#x = Dense(8000, activation='sigmoid')(input_layer)

#x = Dense(4000, activation='sigmoid')(x)

#x = Dense(2000, activation='sigmoid')(x)

#x = Dense(1000, activation='sigmoid')(x)

#x = Dense(500, activation='sigmoid')(input_layer)

x = Dense(100, activation='sigmoid')(input_layer)

x = Dense(50, activation='sigmoid')(input_layer)

#x = GlobalMaxPooling1D()(x)

output_layer = Dense(1, activation='sigmoid')(x)

mlp3 = keras.models.Model(inputs = input_layer, outputs = output_layer)

mlp3.summary()


#compile
mlp3.compile(optimizer=opt, loss=lossType, metrics=['accuracy'])


print()
print("Starting training...")
print()


start = time.time()

# fit the model
mlp3.fit(trainSet, trainLabel, epochs=epochs_to_run, verbose=1)

durationSecond = time.time()-start

print()
print("Training done in",durationSecond/60,"minutes or",durationSecond,"seconds now evaluating...")
print()



# evaluate the model
trainLoss, trainAccuracy = mlp3.evaluate(trainSet, trainLabel, verbose=1)
#print('Training Accuracy: %f' % (accuracy*100))

print()


# evaluate the model
testLoss, testAccuracy = mlp3.evaluate(testSet, testLabel, verbose=1)
#print('Test Accuracy: %f' % (accuracy*100))


word = "dystrophin"

prediction = mlp3.predict(np.array([word2vec(word)]))


print(prediction)


Dense3Results += [trainAccuracy,testAccuracy]

#del model

#backend.clear_session()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 96)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 50)                4850      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 51        
Total params: 4,901
Trainable params: 4,901
Non-trainable params: 0
_________________________________________________________________

Starting training...

Epoch 1/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0663 - accuracy: 0.9790
Epoch 2/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0480 - accuracy: 0.9853
Epoch 3/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0447 - accuracy: 0.9865
Epoch 4/300
355126/355126 [==========

355126/355126 [==============================] - 21s 60us/step - loss: 0.0246 - accuracy: 0.9924
Epoch 142/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0247 - accuracy: 0.9923
Epoch 143/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0247 - accuracy: 0.9923
Epoch 144/300
355126/355126 [==============================] - 21s 59us/step - loss: 0.0245 - accuracy: 0.9924
Epoch 145/300
355126/355126 [==============================] - 21s 59us/step - loss: 0.0245 - accuracy: 0.9923
Epoch 146/300
355126/355126 [==============================] - 21s 59us/step - loss: 0.0246 - accuracy: 0.9925
Epoch 147/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0244 - accuracy: 0.9926
Epoch 148/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0245 - accuracy: 0.9923
Epoch 149/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0243 - accuracy: 0.9926
Epoch 150/300
3

355126/355126 [==============================] - 21s 60us/step - loss: 0.0216 - accuracy: 0.9934
Epoch 288/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0216 - accuracy: 0.9933
Epoch 289/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0216 - accuracy: 0.9934
Epoch 290/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0215 - accuracy: 0.9935
Epoch 291/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0215 - accuracy: 0.9934
Epoch 292/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0216 - accuracy: 0.9935
Epoch 293/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0214 - accuracy: 0.9935
Epoch 294/300
355126/355126 [==============================] - 21s 60us/step - loss: 0.0215 - accuracy: 0.9934
Epoch 295/300
355126/355126 [==============================] - 22s 61us/step - loss: 0.0214 - accuracy: 0.9934
Epoch 296/300
3

In [31]:
from prettytable import PrettyTable

results = PrettyTable()



results.field_names = ["Accuracy",
                       "train",  "test"]#,
                       #"Train 300 epochs", "Test 300 epochs",
                       #"Train 400 epochs", "Test 400 epochs"]

'''
denseResults = [x*100 for x in denseResults]

CNNResults = [x*100 for x in CNNResults]

LSTMResults = [x*100 for x in LSTMResults]
'''




results.add_row(["Dense"] + Dense1Results)

results.add_row(["Dense"] + Dense2Results)

results.add_row(["Dense"] + Dense3Results)

print(results)

+----------+--------------------+--------------------+
| Accuracy |       train        |        test        |
+----------+--------------------+--------------------+
|  Dense   | 0.9920704364776611 | 0.985357403755188  |
|  Dense   | 0.9932981729507446 | 0.9854925274848938 |
|  Dense   | 0.9936163425445557 | 0.9839156866073608 |
+----------+--------------------+--------------------+


<h1>Practical results when applied to our list of proteins</h1>

In [36]:
results = []

for p in proteinVectors:
    results +=[  mlp2.predict(np.array([p]))[0][0] ]

print(len(results),"proteins obtained")
print()

positive = [r for r in results if r>0.5]
print(len(positive),"positive results")

207468 proteins obtained

204869 positive results


In [37]:
import joblib
joblib.dump(mlp2, 'proteinRecognitionNetwork.pkl', compress=9)
print("exported")


exported
